In [1]:
import numpy as np
import cv2

# Stage 1

Stage 1 of this task is concerned with blending an object, taken from one image, onto a scene in another image.

Firstly, the two scenes and the object mask are loaded using *cv2* functions.

**s1** is the scene containing a single object: an ornament elephant.

**s2** is the scene containing two objects: an ornament elephant and a small ornament glass.

**mask** is the object mask for the small ornament glass.

In [2]:
s1 = cv2.imread("Images/1_colour.jpeg", 1) #Scene with 1 object
s2 = cv2.imread("Images/2_colour.jpeg") #Scene with 2 objects
s3 = cv2.imread("Images/3_colour.jpeg")
black = cv2.imread("Images/black.png")
red = cv2.imread("Images/Red.png")
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png") #Target object mask
gorg = cv2.imread("gorg.jpg")

In [16]:
def ExtractObject(S2, ObjectMask):
    """Extracts an object from an image scene based on the mask used"""

    final_im = ObjectMask*S2
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [17]:
def ApplyFilter(ExtractedObject, FilterIndex):
    """Applies convolution on the object using pre-defined image kernels"""

    if FilterIndex==0: #Apply no filter
        FilteredExObject = ExtractedObject
    elif FilterIndex==1:
        kernel = np.ones((5, 5), np.float32)/25
        FilteredExObject = cv2.filter2D(ExtractedObject, -1, kernel)
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        img_yuv = cv2.cvtColor(ExtractedObject, cv2.COLOR_BGR2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        FilteredExObject = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    elif FilterIndex==3:
        FilteredExObject = cv2.medianBlur(ExtractedObject, 5)
        #Will need to define some kernels and use them here
    return FilteredExObject

In [18]:
def ObjectBlender(S1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene S1"""

    alpha = 0.6
    beta = 1.0 - alpha
    BlendingResult = cv2.addWeighted(S1, alpha, FilteredExObject, beta, -60)
    cv2.imshow('Blended', BlendingResult)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return BlendingResult

In [19]:
def CompareResult(BlendingResult, S2, metric):
    """Compares the blended image with the scene with 2 objects"""

    if metric==1: #Sum of Squared Distance Error (SSD)
        error = np.sum((BlendingResult-S2)**2)
    elif metric==2: #Mean Squared Error (MSE)
        error = np.sum((BlendingResult.astype("float") - S2.astype("float")) ** 2)
        error /= float(BlendingResult.shape[0] * BlendingResult.shape[1])
    return error

In [20]:
extracted = ExtractObject(s2, mask)
extracted = ApplyFilter(extracted, 3)

In [21]:
blended = ObjectBlender(s1, extracted)

In [22]:
CompareResult(blended, s2, 2)

1414.880554470486

# Stage 2

Stage 2 of this task is concerned with removing the green background from an image and replacing it with another background.

In [10]:
def RemoveGreen(img):
    """Removes the green background from an image"""

    lower_bound = np.array([47,75,31]) #Upper bound of green colour
    upper_bound = np.array([85,112,48])
    mask = cv2.inRange(img, lower_bound, upper_bound)
#     test = cv2.bitwise_and(img, img, mask = mask)
    test = cv2.bitwise_not(img)

    cv2.imshow("Image without background", test)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [11]:
RemoveGreen(s3)

In [12]:
def NewBackground(imgNoBg, NewBackground):
    """Replaces the background of an image with NewBackground"""

In [4]:
class MaskedImage:
    def __init__(self, mask, image):
        self.mask = mask
        self.image = image

def process_foreground_image(frame, lower_green, upper_green):
    """Create an image mask to change green pixels to black on the foreground image"""
    img = np.copy(frame)

    mask = cv2.inRange(img, lower_green, upper_green)

    masked_image = np.copy(img)
    masked_image[mask != 0] = [0, 0, 0]

    return MaskedImage(mask, masked_image)

def process_background_image(background_frame, frame, mask):
    """Create another image mask to turn non-green pixels black on the background image"""
    background_image = cv2.cvtColor(background_frame, cv2.COLOR_BGR2RGB)

    crop_background = cv2.resize(background_image, (frame.shape[1], frame.shape[0]))
    crop_background[mask == 0] = [0, 0, 0]
    return crop_background

def chroma_key_image(frame, background_image, lower_green=None, upper_green=None):
    """Chroma key image method."""

    lower_green = lower_green if lower_green is not None else np.array([0, 100, 0])
    upper_green = upper_green if upper_green is not None else np.array([80, 255, 40])

    if frame is None or background_image is None:
        raise RuntimeError("Foreground or background image is null.")

    cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
    foreground = process_foreground_image(frame, lower_green, upper_green)
    background = process_background_image(background_image, frame, foreground.mask)
    without_back = np.array(foreground.image + background)
    cv2.imshow("Chroma keyed", without_back)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
lower_bound = np.array([8, 25, 8]) #Upper bound of green colour
upper_bound = np.array([110, 120, 60])
gorg = cv2.cvtColor(gorg, cv2.COLOR_BGR2RGB)
cv2.imshow("Red", gorg)
cv2.waitKey(0)
cv2.destroyAllWindows()
chroma_key_image(frame = s3, background_image = gorg, lower_green=lower_bound, upper_green=upper_bound)